In [1]:
from nltk import word_tokenize
from pyvi import ViTokenizer, ViPosTagger
import random

In [2]:
# Load danh sách các từ stop word
stoplist = []

with open("./data/vnstopword.txt", encoding="utf-8") as f :
  text = f.read()
  for word in text.split('\n') :  #Tách ra mỗi dòng là 1 từ stopword riêng lẻ
      stoplist.append(word)
  f.close()

In [3]:
from gensim.models import KeyedVectors, Word2Vec

# Load mô hình trained word2vec để embedding
word2vec_model_path = ("./data/w2v.bin")

#w2v = Word2Vec.load(word2vec_model_path)
w2v = KeyedVectors.load_word2vec_format(word2vec_model_path,binary = True)

In [5]:
# Hàm hoán đổi ngẫu nhiên vị trí các từ trong câu. Số lượng câu mới tạo ra là n

def Random_Swap(sentence,n):
    new_sentences = []
    words = sentence.split()
    for i in range(n):
        random.shuffle(words)
        new_sentences.append(' '.join(words))
    return new_sentences


In [6]:
# Hàm xoá từ trong câu nếu từ đó là: Động từ (V), giới từ (C), số (M), dấu câu (F)

def Random_Deletion(sentence):
    
    new_sentence = []
    tagged_word = ViPosTagger.postagging(sentence)
    
    # Chia POS thành 2 list cho dễ thực hiện
    word = tagged_word[0]
    tag = tagged_word[1]

    edited_sentence = [i for i,j in zip(word,tag) if j != 'P' and j != 'V' and j != 'C' and j != 'F' and j != 'M']
    edited_sentence = ' '.join(edited_sentence)
    new_sentence.append(edited_sentence)
    return new_sentence


In [7]:
# Hàm thay thế từ đồng nghĩa

def syn_rep(sentence, synonyms_lexicon):
    keys = synonyms_lexicon.keys()
    words = sentence.split()
    n_sentence = sentence
    for w in words:
        if w not in stoplist:
            if w in keys:
                n_sentence = n_sentence.replace(w, synonyms_lexicon[w][0])  # Thay đổi từ đồng nghĩa ở cột kế tiếp
    return n_sentence

#===================================================================================

def Synonym_Replacement(sentence):
    #Get synonyms word from this file
    synonyms_lexicon = get_synonyms('./data/vietnamsyn.txt')
    
    new_sentence = []
    
    sen_replaced = syn_rep(sentence, synonyms_lexicon)
    new_sentence.append(sen_replaced)
    
    return new_sentence

In [8]:
# Hàm lấy từ đồng nghĩa trong file vietnamsyn.txt
#synonyms_lexicon = get_synonyms('./vietnamsyn.txt')

def get_synonyms(path):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path, encoding="utf8").readlines()]
    for e in text_entries:
        e = e.split('\t')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

#===================================================================================
def Insert(sentence, synonyms_lexicon):
    keys = synonyms_lexicon.keys()
    words = sentence.split()
    n_sentence = sentence
    for w in words:
        if w not in stoplist:
            if w in keys:
                n_sentence = n_sentence + ' ' + synonyms_lexicon[w][0] # Chèn từ đồng nghĩa vào cuối câu.
    return n_sentence

#===================================================================================

def Random_Insert(sentence):
    #Get synonyms word from this file
    synonyms_lexicon = get_synonyms('./data/vietnamsyn.txt')
    
    new_sentence = []
    
    sen_inserted = Insert(sentence, synonyms_lexicon)
    new_sentence.append(sen_inserted)
    
    return new_sentence

In [9]:
# Finding out a similarity word from word embedding space

def Similarity(word):
    # Lấy ra 1 similarity word đầu tiên với score lớn nhất
    word_similarity = w2v.most_similar(word,topn=1)
    
    # Vòng lặp này để trả về word đầu tiên, với similarity score là lớn nhất
    for x in word_similarity:
        word = ''.join(x[0]) # Lệnh này lấy ra chữ, bỏ qua score
    return word   

#===================================================================================
# Repalcement similitary word from word vector embedding space

def Word_Replacement(sentence):
    words = sentence.split()
    replaced_sentence = []
    new_sentence = ''
    for word in words:
        if word not in w2v:
            new_sentence = new_sentence + ' ' + word
        else: 
            if word in stoplist:
                new_sentence = new_sentence + ' ' + word
            else:
                new_word = Similarity(word)
                new_sentence = new_sentence + ' ' + new_word
                
    replaced_sentence.append(new_sentence.strip())
    return replaced_sentence


In [10]:
# Load file data to augmentate
from sklearn.datasets import load_files

training_data = load_files(r"./data2", encoding="utf-8")
X_train, y_train = training_data.data, training_data.target


In [11]:
# Số comment, 3 pos, 3 neg
#print(X_train[1],y_train[1])

In [12]:
# Đoạn code này là đọc tất cả các comment và label. Thực hiện agumentation.
# Sau đó lưu vào file .csv
import csv   
header=['comment','label']
with open(r'data2.csv', 'a', encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in range(0,len(X_train)):
        comment = str(X_train[i])
        label = y_train[i]
        data = [comment,label]
        writer.writerow(data)
        
        # Thực hiện swapping
        data= [Random_Swap(comment,1),y_train[i]]
        writer.writerow(data)
    
        
        # Thực hiện việc xoá ngẫu nhiên
        data= [Random_Deletion(comment),y_train[i]]
        writer.writerow(data)
        
        # Thực hiện việc chèn từ đồng nghĩa cuối câu
        data= [Random_Insert(comment),y_train[i]]
        writer.writerow(data)
        
        # Thực hiện việc thay thế từ gần nghĩa lấy từ word embedding
        data= [Word_Replacement(comment),y_train[i]]
        writer.writerow(data)
        
        # Thực hiện thay thế từ đồng nghĩa lấy từ file từ đồng nghĩa
        #data= [Synonym_Replacement(comment),y_train[i]]
        #writer.writerow(data)

In [13]:
# sau khi augment xong lưu nội dung vào file .csv

import pandas as pd
a = pd.read_csv('data2.csv', encoding='utf8')

In [18]:
a.shape

(120000, 2)